In [1]:
import pandas as pd
import time
import nibabel as nb
from nilearn import plotting as plot
from nilearn import image as img
import matplotlib.pyplot as plt
import numpy as np
from scipy import ndimage as nd
from scipy import stats

In [2]:
df = pd.read_excel("ATR_GT_Training_og.xlsx",header = None, names=['File Name','Label'])
df['File Name'] = df['File Name'].map(lambda x: x.lstrip("'").rstrip("'"))
df['Image'] = df['File Name'].map(lambda x: nb.load('ATR_data/' + str(x) + '.nii.gz'))

In [3]:
feature_stats = np.load('stat_features.npy')
feature_histogram = np.load('features_histogram.npy')
features = np.concatenate((feature_stats,feature_histogram),axis=1)

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = features[:1005]
y = df.Label[:1005]

#standarize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [5]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.model_selection import KFold
from sklearn.svm import SVC

knn = KNeighborsClassifier(n_neighbors = 1, leaf_size = 1, p=1)
svm = SVC(kernel='linear', C=5)

In [6]:
#Returns overall Accuracy Score
def overall_acc(y_true,y_pred):
    return accuracy_score(y_true,y_pred)

#Returns the percent detected(#targets detected and #targets)
def PD(y_true,y_pred):
    return recall_score(y_true,y_pred)

#Returns the percent of false alarms(#false alarms and #non-targets)
def PFA(y_true,y_pred):
    return 1 - accuracy_score(1-y_true,1-y_pred)

In [7]:
def results(X,y,clf):
    kf = KFold(n_splits=5)
    per_det_total = []
    per_det_0 = []
    per_det_1 = []
    per_det_2 = []
    per_det_3 = []
    per_false_alarm = []
    acc = []
    
    
    for train_i, test_i in kf.split(X):
        X_train, X_test = X[train_i],X[test_i]
        y_train, y_test = y[train_i], y[test_i]
        
        predict = clf.fit(X_train, y_train != 0).predict(X_test)
        
        acc.append(overall_acc(y_test != 0, predict))
        per_det_total.append(PD((y_test != 0),predict))
        per_det_0.append(PD((y_test == 0),predict))
        per_det_1.append(PD((y_test == 1),predict))
        per_det_2.append(PD((y_test == 2),predict))
        per_det_3.append(PD((y_test == 3),predict))
        per_false_alarm.append(PFA(y_test != 0, predict))
        
        
    print('Accuracy: ', np.mean(acc))
    print('Percent False Alarm: ', np.mean(per_false_alarm))
    print('Percent Total Detected: ', np.mean(per_det_total))
    print('Percent Non-Targets Detected: ',np.mean(per_det_0))
    print('Percent Saline Detected: ', np.mean(per_det_1))
    print('Percent Rubber Detected: ', np.mean(per_det_2))
    print('Percent Clay Detected: ', np.mean(per_det_3))

In [8]:
results(X_scaled, y, knn)

Accuracy:  0.8139303482587066
Percent False Alarm:  0.18606965174129353
Percent Total Detected:  0.6937145271118196
Percent Non-Targets Detected:  0.1353264015728631
Percent Saline Detected:  0.6363095238095238
Percent Rubber Detected:  0.6500925079448
Percent Clay Detected:  0.823992673992674


C:\Users\Cassa\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\Cassa\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdim

In [28]:
results(X_scaled,y,svm)

Accuracy:  0.7074626865671642
Percent False Alarm:  0.29253731343283584
Percent Total Detected:  0.2862995619311577
Percent Non-Targets Detected:  0.09889651636176726
Percent Saline Detected:  0.025
Percent Rubber Detected:  0.39078947368421046
Percent Clay Detected:  0.5176984126984128
